In [1]:
import openpathsampling as paths
from openpathsampling.visualize import PathTreeBuilder
from IPython.display import SVG

In [2]:
st = paths.storage.Storage('toy_retis.nc', mode='r')

In [3]:
psi = st.cvs[0]
print psi.name
print [ cv.name for cv in st.cvs ]

xval
[u'xval']


In [48]:
tree = PathTreeBuilder(st)
tree.rejected = True
# Some ideas for collectivevariables to visualize
tree.op = lambda snap : 'B' if snap.reversed else 'F'
tree.op = st.configurations.op_idx
tree.from_samples(tree.construct_heritage(st, st.samples[139]))

ValueError: min() arg is an empty sequence

In [43]:
view = tree.renderer
view.zoom = 1.1
view.scale_y = 24
view.scale_x = 12
view.font_size = 0.35
SVG(view.to_svg())

In [44]:
cvA = st.cvs['StateA']
cvB = st.cvs['StateB']
cvX = st.cvs['StateX']

ValueError: str "StateA" not found in storage

In [34]:
st.samples.count()

144

In [45]:
samps = []
for sampset in st.samplesets:
    samps.extend([sa for sa in sampset if sa.replica == 1])

In [53]:
samps[-1].parent

<Sample @ 0x10a765c50>